In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import common as cm

# Part 1: Information Gain

Important note: this exercise uses Pandas (for data manipulation and analysis) and Graphviz (for graph-drawing) libraries. 

This exercise consists of 3 parts. Complete the first part to get a mark of 3.0, the first two parts to get 4.0, complete all assignments to get 5.0. 

1.1 ) There are 10 objects (data) characterized with 5 binary attributes:

In [2]:
attributeNames = ["attr 1", "attr 2", "attr 3", "attr 4", "attr 5"]

data = pd.DataFrame(
    [
        [1, 0, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1],
    ],
    columns=attributeNames,
)

1.2) Each object is assigned to either a class "0" or "1". The assignments are as follows (cl):

In [3]:
data["cl"] = [1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

Hint: How one can read data (columns) in Pandas:

In [4]:
print(data["cl"])
print(list(data["cl"]))
print(set(data["cl"]))
print(data["attr 1"])

0    1
1    1
2    1
3    0
4    0
5    1
6    1
7    1
8    0
9    0
Name: cl, dtype: int64
[1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
{0, 1}
0    1
1    1
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    0
Name: attr 1, dtype: int64


Hint: How split data (Pandas DataFrame) base on column:

In [5]:
data[data['cl']==0]

,attr 1,attr 2,attr 3,attr 4,attr 5,cl
3,1,0,1,0,1,0
4,1,0,0,1,1,0
8,0,1,0,0,1,0
9,0,0,0,1,1,0


Hint: How take values from column (Pandas Series):

In [6]:
for id, row in data['cl'].items():
    print(id,row)

0 1
1 1
2 1
3 0
4 0
5 1
6 1
7 1
8 0
9 0


1.3 )  Finish the below function for calculating entropy. $H(CL) = - \sum_{y \in CL}p(y)log_2p(y)$ It should return a value of entropy for an input vector CL. Assume that $log_2(0)$ is equal to 0.

In [7]:
def getEntropy(cl):
    entropy = 0
    classes = cl.value_counts()
    elements = cl.count()
    for y in classes:
        p = y / elements
        if p != 0:
            entropy -= p * math.log(p, 2)
    return entropy
        

1.4 ) Calculate the entropy for the CL vector:

In [8]:
print(getEntropy(data['cl']))

0.9709505944546686


1.5) Finish the below function for calculating a conditional entropy: $H(CL|X) = - \sum_{x \in X} \sum_{y \in CL} p(x,y) log_2 \frac{p(x,y)}{p(x)}$. Assume that $log_2(0)$ is equal to 0 and if $p(x) = 0$, $\frac{p(x,y)}{p(x)}$ is equal to 0 as well.

In [9]:
def getConditionalEntropy(cl, attr):
    entropy = 0
    data = pd.DataFrame({'cl': cl, 'attr': attr})
    elements = data.cl.count()
    X_classes = set(data.attr)
    Y_classes = set(data.cl)
    for x in X_classes:
        x_data = data[data.attr == x]
        x_elements = x_data.cl.count()
        p_x = x_elements / elements
        if p_x != 0:
            for y in Y_classes:
                y_elements = x_data[x_data.cl == y].cl.count()
                p_y = y_elements / x_elements
                if p_y != 0:
                    entropy -= p_x * p_y * math.log(p_y, 2)
    return entropy
        

1.6 ) Calculate conditional entropies for given attribiutes.

In [10]:
print(getConditionalEntropy(data["cl"], data["attr 1"]))
print(getConditionalEntropy(data["cl"], data["attr 5"]))
### print(getConditionalEntropy(pd.Series([1, 0, 1, 0, 0, 1, 0, 0]), pd.Series([1, 1, 0, 0, 0, 1, 0, 1])))
### print(getConditionalEntropy(pd.Series([1, 1, 0, 0, 1]), pd.Series([1, 0, 0, 0, 1])))
### print(getConditionalEntropy(pd.Series([0, 0, 0]), pd.Series([ 1, 1, 1])))

0.9509775004326937
0.9709505944546686


1.7 ) Which entropy is lesser and why?

Entropy for attribute 1 is lesser than for attribute 5, because the latter introduces no new information (all values are equal to one) and it's entropy is equal to 'general' entropy.

1.8) Finish the below function for calculating information gain:

In [11]:
def getInformationGain(cl, attr):
    H_y = getEntropy(cl)
    H_x = getConditionalEntropy(cl, attr)
    return H_y - H_x

In [12]:
print(getInformationGain(data["cl"], data["attr 1"]))
print(getInformationGain(data["cl"], data["attr 5"]))

0.01997309402197489
0.0


# Part 2: ID3 algorithm

Decision tree consists of decision nodes and leaves. Nodes split data while leaves classify objects. Consider the class "Node" provided below. It consists of 4 fields:
- attr - attribute ID (use the names in attributeNames vector)
- left - left branch, i.e., a reference to other node
- right - right branch, i.e., a reference to other node
- value - a decision. If node = None, then the node is not a leaf. If value is not None, then a node is considered a leaf. 

Method __call__ returns the decision if the node is a leaf (i.e., when value is not None). 
Otherwise, it calls either the left or the right branch of an input object, based on the attribute value (0 -> left children; 1 -> right children). In this way, we can traverse the decision tree in order to find the final decision.

In [13]:
class Node:
    def __init__(self, attr, left, right, value):
        self.attr = attr
        self.left = left
        self.right = right
        self.value = value

    def __call__(self, obj):
        if self.value is None:
            if obj[self.attr] == 0:
                return self.left(obj)
            else:
                return self.right(obj)
        else:
            return self.value
        
### EXAMPLE
def example(obj):
    root = Node(0, None, None, None)
    lChildren = Node(1, None, None, None)
    rChildren = Node(None, None, None, 2)
    root.left = lChildren
    root.right = rChildren
    llChildren = Node(None, None, None, 3)
    rrChildren = Node(None, None, None, 4)
    lChildren.left = llChildren
    lChildren.right = rrChildren
    print(root(obj))
    
example([0, 0])
example([0, 1])
example([1, 0])
example([1, 1])

3
4
2
2


2.1) Create an initial root. Set the value (decision) to 1. 

In [14]:
root = Node(None, None, None, 1)

2.2) Use a getErrorRate method in common.py auxiliary file to calculate the error rate. The decision is made based on the majority rule. In case of tie, the method takes 0 as the default class.

In [15]:
cm.getErrorRate(root, data)

0.4

2.3) Use printGraph method (see the common.py file) to draw the decision tree and save it in a png file.

In [16]:
cm.printGraph(root)

2.4) Calculate information gain for all attribiutes.

In [17]:
def printInformationGain(data):
    for attribute_name in attributeNames:
        IG = getInformationGain(data['cl'], data[attribute_name])
        print('{}: {}'.format(attribute_name, IG))
        
printInformationGain(data)

attr 1: 0.01997309402197489
attr 2: 0.0464393446710154
attr 3: 0.12451124978365313
attr 4: 0.09127744624168
attr 5: 0.0


2.5) Choose the best attribute to split the data. Construct two new nodes: one for $x_i$ = 0 decision and the second for $x_i$ = 1; connect them with the root (left and right branch). Remember to update the root. 

In [18]:
### attribute 3 provides the best Information Gain

### print(data[data['attr 3'] == 0])
### majority class = 0

### print(data[data['attr 3'] == 1]) 
### majority class = 1

LChild = Node(None, None, None, 0)
RChild = Node(None, None, None, 1)

root.attr = 'attr 3'
root.left = LChild
root.right = RChild
root.value = None

2.6) Print the graph and calculate the error rate. What happened with the error rate?

In [19]:
cm.printGraph(root)
cm.getErrorRate(root, data)

### Error rate decreased from 40% --> 30%

0.30000000000000004

2.7) Split the 'data' (table) based on the selected attribiute, i.e., create two new tables.

In [20]:
Ldata = data[data['attr 3'] == 0]
Rdata = data[data['attr 3'] == 1]

2.8) Let us start with the left node. Firstly, calculate information gain for this node.

In [21]:
printInformationGain(Ldata)

attr 1: 0.4199730940219748
attr 2: 0.01997309402197489
attr 3: 0.0
attr 4: 0.01997309402197489
attr 5: 0.0


2.9) Choose the best attribute to split the data and then update the decision tree.

In [22]:
### Best atrribute - attr 1 

### print(Ldata[Ldata['attr 1'] == 0])
### majority class = 0

### print(Ldata[Ldata['attr 1'] == 1]) 
### majority class = 1

LLChild = Node(None, None, None, 0)
LRChild = Node(None, None, None, 1)

LChild.attr = 'attr 1'
LChild.left = LLChild
LChild.right = LRChild
LChild.value = None


2.10) Print the graph and calculate the error rate. What happened with the error rate?

In [23]:
cm.printGraph(root)
cm.getErrorRate(root, data)

0.19999999999999996

2.11) Split data (remember that we split left_data, not data).

In [24]:
LLdata = Ldata[Ldata['attr 1'] == 0]
LRdata = Ldata[Ldata['attr 1'] == 1]

2.12) Repeat the whole process for the right node.

In [25]:
printInformationGain(Rdata)

attr 1: 0.17095059445466854
attr 2: 0.17095059445466854
attr 3: 0.0
attr 4: 0.7219280948873623
attr 5: 0.0


In [26]:
### Best attribute - attr 4

### print(Rdata[Rdata['attr 4'] == 0])
### majority class = 0

### print(Rdata[Rdata['attr 4'] == 1]) 
### majority class = 1

RLChild = Node(None, None, None, 0)
RRChild = Node(None, None, None, 1)

RChild.attr = 'attr 4'
RChild.left = RLChild
RChild.right = RRChild
RChild.value = None


In [27]:
cm.printGraph(root)
cm.getErrorRate(root, data)

0.09999999999999998

In [28]:
RLdata = Rdata[Rdata['attr 4'] == 0]
RRdata = Rdata[Rdata['attr 4'] == 1]

2.13) Let's consider left-left node. Calculate information gain for it.

In [29]:
printInformationGain(LLdata)

attr 1: 0.0
attr 2: 0.0
attr 3: 0.0
attr 4: 0.0
attr 5: 0.0


2.14) Will adding a new node to the tree improve its effectiveness? Why? Why not?

No, because we gain no new information with addition of next nodes.

2.15) Calculate information gain for the left-right node.

In [30]:
printInformationGain(LRdata)

attr 1: 0.0
attr 2: 0.2516291673878229
attr 3: 0.0
attr 4: 0.2516291673878229
attr 5: 0.0


In [31]:
### Attr 2 and 4 are equal - let's split on attr 2

### print(LRdata[LRdata['attr 2'] == 0]) 
### no majority class - we take majority class of LRChild (1)

### print(LRdata[LRdata['attr 2'] == 1]) 
### majority class = 1

LRLChild = Node(None, None, None, 1)
LRRChild = Node(None, None, None, 1)

LRChild.attr = 'attr 2'
LRChild.left = LRLChild
LRChild.right = LRRChild
LRChild.value = None

In [32]:
cm.printGraph(root)
cm.getErrorRate(root, data)

0.09999999999999998

2.16) What happened with the error rate? Is it necessary to keep these two newly added leaves?

Error rate did not decrease, therefore these newly added leaves are unnecessary as they provide no new valuable information.

In [33]:
### We can revert changes made in previous step as it does not increase accuracy of our prediction

LRChild.attr = None
LRChild.left = None
LRChild.right = None
LRChild.value = 1

cm.printGraph(root)
cm.getErrorRate(root, data)

0.09999999999999998

2.17) Finish creating the right side of the tree

In [34]:
printInformationGain(RLdata)
### no further split is necessary

attr 1: 0.0
attr 2: 0.0
attr 3: 0.0
attr 4: 0.0
attr 5: 0.0


In [35]:
printInformationGain(RRdata)
### no further split is necessary either

attr 1: 0.0
attr 2: 0.0
attr 3: 0.0
attr 4: 0.0
attr 5: 0.0


In [43]:
### Final Tree:
cm.printGraph(root, data, fileName = 'Part2')
cm.getErrorRate(root, data)

print(LLdata)
print(LRdata)
print(RLdata)
print(RRdata)

   attr 1  attr 2  attr 3  attr 4  attr 5  cl
8       0       1       0       0       1   0
9       0       0       0       1       1   0
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
1       1       1       0       0       1   1
4       1       0       0       1       1   0
7       1       0       0       1       1   1
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
3       1       0       1       0       1   0
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
0       1       0       1       1       1   1
2       0       1       1       1       1   1
5       0       0       1       1       1   1
6       1       1       1       1       1   1


# Part 3: automated construction of decision trees

3.1 Complete the following function for automated construct of decision trees, so that it returns a decision tree for the given data and attribute list. Note that this is a recusive method, i.e., calls itself.

In [38]:
max_depth = 0

def createTree(data, attributeNames,par_atr ,depth=0):
    data = data.reset_index().drop("index", axis=1)
    new_root = Node(None,None,None,par_atr)
    attr_ratio = []
    for attribute_name in attributeNames:
        IG = getInformationGain(data['cl'],data[attribute_name])
        attr_ratio.append([attribute_name,IG])
    best_attr = max(attr_ratio, key = lambda x: x[1])
    max_0 = 0
    max_1 = 0
    for i in data['cl']:
        if i == 0:
            max_0 += 1
        else:
            max_1 += 1
    if(max_0<max_1):
        new_root.value = 1
    else:
        new_root.value = 0
    if depth <= max_depth and best_attr[1] != 0:
        new_root.attr = best_attr[0]
        new_root.value = None
        
        new_left = Node(None,None,None,None)
        new_right = Node(None,None,None,None)
        
        left_data = data[data[best_attr[0]] == 0]
        right_data = data[data[best_attr[0]] == 1]
        
        depth += 1
        
        new_root.left = createTree(left_data,attributeNames,0,depth)
        new_root.right = createTree(right_data,attributeNames,1,depth)
    return new_root
    
cm.printGraph(createTree(data,attributeNames,None),data, fileName='Part3_Example')  

3.2) Build a decision tree for a training dataset in the common.py auxiliary file, for diffrent values of max_depth.  Calculate & compare the error rates for training and validation datasets.

In [39]:
max_depth = 10

In [40]:
# Training dataset
train_attributeNames, train_data = cm.getTrainingDataSet()
cm.printGraph(createTree(train_data,train_attributeNames,None),train_data, fileName = 'Part3_Train')
cm.getErrorRate(createTree(train_data,train_attributeNames,None),train_data)

0.19999999999999996

In [41]:
# Validation dataset
valid_attributesName, valid_data = cm.getValidationDataSet()
cm.printGraph(createTree(train_data, valid_attributesName, None),valid_data, fileName = 'Part3_Validation')
cm.getErrorRate(createTree(train_data,valid_attributesName,None),valid_data)

0.30000000000000004

3.3) Consider only the training data set and answer the following questions:
* What is the miximum depth of the tree (consider only the training data set)?
* The tree building process should stop when there is no improvement in error rate (why?). Check for which value of "max_dept" there is no improvement in error rate. 

In [42]:
err_rate = 1
best_max_depth = 0
for i in range(10):
    max_depth = i
    x = cm.getErrorRate(createTree(train_data,train_attributeNames,None),train_data)
    if x < err_rate:
        err_rate = x
        best_max_depth = i
        continue
    else:
        max_depth = best_max_depth
        cm.printGraph(createTree(train_data,train_attributeNames,None),train_data, fileName = 'Part3_BestTree')
        break